In [1]:
import arrow
import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import os
import pandas as pd
import pickle
import xarray as xr

%matplotlib inline

## The Grid ##

In [ ]:
mesh = nc.Dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')
gdepw = mesh.variables['gdepw_1d'][0]
bathy = nc.Dataset('/home/sallen/MEOPAR/grid/bathymetry_201702.nc')
lats = bathy.variables['nav_lat'][:]
lons = bathy.variables['nav_lon'][:]
mesh.close()
bathy.close()

## Functions ##

In [ ]:
def calculate_transport(basedir, dir1, section):
    filename = 'ariane_positions_quantitative.nc'
    with nc.Dataset(os.path.join('/data/sallen/results/Ariane',basedir, dir1, filename)) as dataset:
#    with nc.Dataset(os.path.join('../',basedir, dir1, filename)) as dataset:
        final_section = dataset.variables['final_section'][:]
        final_transport = dataset.variables['final_transp'][:]
        vic_transport = np.sum(np.where(final_section == section, final_transport, 0))
    return (vic_transport/24.)

In [13]:
def calculate_all(basedir, dir1, section):
    filename = 'ariane_positions_quantitative.nc'
    with nc.Dataset(os.path.join('/data/sallen/results/Ariane',basedir, dir1, filename)) as dataset:
#    with nc.Dataset(os.path.join('../',basedir, dir1, filename)) as dataset:
        final_section = dataset.variables['final_section'][:]
        final_transport = dataset.variables['final_transp'][:]
        final_age = dataset.variables['final_age'][:]/86400. # convert from seconds to days
        final_depth 
        vic_transport = np.sum(np.where(final_section == section, final_transport, 0))
        mean_age = np.sum(np.where(final_section == section, final_transport*final_age, 0))/vic_transport
    return vic_transport/24., mean_age

In [ ]:
def get_data(start, endtime, basedir, section):
    transport = np.zeros(365*2+1)
    timerange = arrow.Arrow.range('day', start, endtime)
    time = []
    for i, r in enumerate(timerange):
        dir1 = r.format('DDMMMYY').lower()
        transport[i] = calculate_transport(basedir, dir1, section)
#        transport[i] = calculate_transport('./', './', section)
        time.append(r.datetime)
    return time, transport[:len(timerange)]  

In [5]:
def get_data_all(start, endtime, basedir, section):
    transport = np.zeros(365*2+1)
    age = np.zeros(365*2+1)
    timerange = arrow.Arrow.range('day', start, endtime)
    time = []
    for i, r in enumerate(timerange):
        dir1 = r.format('DDMMMYY').lower()
        transport[i], age[i] = calculate_all(basedir, dir1, section)
#        transport[i] = calculate_transport('./', './', section)
        time.append(r.datetime)
    return time, transport[:len(timerange)], age[:len(timerange)]

In [ ]:
def save_position(basedir, dir1, section, fi, ff):
    print(fi, ff)
    filename = 'ariane_positions_quantitative.nc'
#    with nc.Dataset(os.path.join('/data/sallen/results/Ariane',basedir, dir1, filename)) as dataset:
    with nc.Dataset(os.path.join('../',basedir, dir1, filename)) as dataset:
        final_section = dataset.variables['final_section'][:]
        final_transport = dataset.variables['final_transp'][:]
        final_depth = dataset.variables['final_depth'][:]
        final_lon = dataset.variables['final_lon'][:]
        final_lat = dataset.variables['final_lat'][:]
        init_depth = dataset.variables['init_depth'][:]
        init_lat = dataset.variables['init_lat'][:]
        init_lon = dataset.variables['init_lon'][:]
        for part in range(final_section.shape[0]):
            if final_section[part] == section:
                fi.write('{0:4f} {1:4f} {2:4f} {3:4f} \n'.format(
                    init_depth[part], init_lat[part], init_lon[part], final_transport[part]))
                ff.write('{0:4f} {1:4f} {2:4f} {3:4f} \n'.format(
                    final_depth[part], final_lat[part], final_lon[part], final_transport[part]))
    

## Get the Data ##

In [15]:
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 1, 4)
time, transport, dayage = get_data_all(start, endtime, 'FullSouth', 2)
fullsouth_transport = pd.DataFrame(data=transport, index=time, columns=['transport'])
fullsouth_all = fullsouth_transport.assign(age=pd.Series(dayage).values)
fullsouth_all
#time, transport = get_data(start, endtime, 'FullNorth', 2)
#fullnorth_transport = pd.DataFrame(data=transport, index=time, columns=['transport'])

,transport,age
2015-01-01 00:00:00+00:00,13075.300257,18.636380
2015-01-02 00:00:00+00:00,13597.997233,19.018793
2015-01-03 00:00:00+00:00,13142.763977,19.628079
2015-01-04 00:00:00+00:00,15892.022729,19.514940


In [6]:
start = datetime.datetime(2016, 7, 1)
endtime = datetime.datetime(2016, 7, 31)
#time, transport = get_data(start, endtime, 'FullSouth', 2)
#fullsouth_transport_16 = pd.DataFrame(data=transport, index=time, columns=['transport'])
time, transport = get_data(start, endtime, 'FullNorth', 2)
fullnorth_transport_16 = pd.DataFrame(data=transport, index=time, columns=['transport'])

NameError: name 'get_data' is not defined

In [ ]:
fi = open('south_init.txt', 'a')
ff = open('south_final.txt', 'a')
save_position('./', './', 2, fi, ff)
fi.close() 
ff.close()

In [ ]:
#Victoria_sill_j = 178-1;
#    Victoria_sill_i = np.arange(235,302+1)-1
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
with open('south_init.txt', 'r') as fi:
    mydata = np.loadtxt(fi)
my_lats_edges = lats[235-1:302-1, 178-1] - (lats[235, 178] - lats[234, 178])/2.
my_deps_edges = -gdepw[28::-1]
H, xs, ys = np.histogram2d(mydata[:, 0], mydata[:, 1],  bins = [my_deps_edges, my_lats_edges],
                        weights=mydata2[:,3])
mesh = ax[0].pcolormesh(my_lats_edges[:-1], my_deps_edges[:-1], H)
fig.colorbar(mesh, ax=ax[0])

with open('south_final.txt', 'r') as ff:
    mydata2 = np.loadtxt(ff)
my_lons_edges = lons[387-1, 262-1:309-1] - (lons[387, 300] - lons[387, 301])/2.
H, xs, ys = np.histogram2d(mydata2[:, 0], mydata2[:, 2],  bins = [my_deps_edges, my_lons_edges],
                        weights=mydata2[:,3])
mesh = ax[1].pcolormesh(my_lons_edges[:-1], my_deps_edges[:-1], H)
fig.colorbar(mesh, ax=ax[1])


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 6))
ax2 = ax.twinx()
fullsouth_transport.plot(ax=ax, y='transport', label="South to North", legend=False)
fullnorth_transport.plot(ax=ax, y='transport', label="North to South (shallow)", legend=False)
fullsouth_transport_16.plot(ax=ax, y='transport', label="South to North (deep)", legend=False, c='b')
fullnorth_transport_16.plot(ax=ax, y='transport', label="North to South (shallow)", legend=False, c='g')
#low_pass_tide.plot(ax=ax2, c='r')
ax.set_xlim(fullsouth_transport.index[0], time[-1])
ax.legend()

## Calculation of Tides moved to "Calculate Tides" ##
Here just read them

In [ ]:
low_pass_tide = pd.read_csv('low_pass_tide.csv')
low_pass_tide.plot()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = 0
ii, jj = 0-min(0,lag), 290-min(0,lag)
ax.plot(range(ii, jj), fullnorth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = 0
ii, jj = 150-min(0,lag), 290-min(0,lag)
ax.plot(range(ii, jj), fullnorth_transport['transport'][ii+lag:jj+lag], 'o-')
ax2.plot(range(ii, jj), low_pass_tide['uVelocity'][ii:jj], 'r')

## Summer Deep Water Variability ##

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = -3
ii, jj = 190, 300
ax.plot(range(ii, jj),fullsouth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

## Spring Deep Water Variability ##

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = -3
ii, jj = 100, 190
ax.plot(range(ii, jj),fullsouth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

## Winter Deep Water Variability : note change in lag

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = -5
ii, jj = 0-lag, 100
ax.plot(range(ii, jj),fullsouth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

In [ ]:
mesh = nc.Dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')

In [ ]:
tracers = nc.Dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DTracerFields1hV17-02')
iY = 258; iX = 178; iZ = 28
iY2 = 388; iX2 = 271
salinity = tracers.variables['salinity'][0:24*365:24, :iZ, iY, iX]
salinity2 = tracers.variables['salinity'][0:24*365:24, :iZ, iY2, iX2]

In [ ]:
e3t = mesh.variables['e3t_0'][0, :iZ, iY, iX]

In [ ]:
saldiff = np.zeros(365)
for i in range(365):
    saldiff[i] = np.sum(salinity[i]*e3t) - np.sum(salinity2[i]*e3t)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 5))
ax2 = ax.twinx()
ax.plot(saldiff)
ax2.plot(range(0, 290), fullsouth_transport['transport'][0:290], 'g')

In [ ]:
fs = open('gulfisland_fullsouth_early2015', 'wb')
pickle.dump(gulfislands, fs)
fs.close()

In [ ]:
fs = open('fullsouth_early2015', 'wb')
pickle.dump(fullsouth_transport, fs)
fs.close()

In [ ]:
vic_transport_2015 = pickle.load('vic_transport_2015')
plt.plot(vic_transport_2015)

In [ ]:
fs = open('vic_transport_2015', 'wb')
pickle.dump(vic_transport, fs)

In [ ]:
vsq = pd.read_csv('../day_avg_tide_pd.csv')

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(18, 5))
for i, lag in enumerate(range(-7, 7, 3)):
    ax[i].plot(vsq[99+lag:99+365+lag].uVelocity, vic_transport, '*')
#    print (np.correlate(vsq[99+lag:99+365+lag], vic_transport[:]), lag, mode='valid')

In [ ]:
print (final_section[:10])

In [ ]:
print (final_transport[:10])